<a href="https://colab.research.google.com/github/lulibpiyu/Primera-Pre-Entrega-Piyuka/blob/main/Primera_Pre_Entrega%2BPiyuka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
from google.colab import drive
import json
import os

drive.mount('/content/drive')
json_path = '/content/drive/My Drive/Primera Pre-Entrega Piyuka/usuarios_db.json'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
# Recuperar BD desde un archivo
def cargar_datos():
    if os.path.exists(json_path):
        with open(json_path, "r") as file:
            return json.load(file)
    return {}

# Guardar BD en un archivo
def guardar_datos(usuarios_db):
    with open(json_path, "w") as file:
        json.dump(usuarios_db, file)

# Diccionario de usuarios, contraseñas y estado de bloqueo
usuarios_db = cargar_datos()
usuario_actual = None

In [32]:
def crear_usuario():
    nombre_usuario = input("Ingrese el nombre de usuario: ")
    if nombre_usuario in usuarios_db:
        print("El usuario ya existe. Intente otro nombre de usuario.")
    else:
        contraseña = input("Ingrese la contraseña: ")
        es_admin = input("¿Es administrador? (s/n): ").lower() == 's'
        usuarios_db[nombre_usuario] = {'contraseña': contraseña, 'intentos': 0, 'bloqueado': False, 'admin': es_admin}
        guardar_datos(usuarios_db)
        print(f"Usuario '{nombre_usuario}' creado exitosamente.")

In [33]:
def login_usuario():
    global usuario_actual
    nombre_usuario = input("Ingrese su nombre de usuario: ")

    if nombre_usuario not in usuarios_db:
        print("El usuario no existe.")
        return

    if usuarios_db[nombre_usuario]['bloqueado']:
        print("Usuario Bloqueado. Contacte al Administrador.")
        return

    intentos_restantes = 3 - usuarios_db[nombre_usuario]['intentos']
    while intentos_restantes > 0:
        contraseña = input("Ingrese su contraseña (o salir para cancelar): ")
        if contraseña.lower() == 'salir':
            print("Proceso de login cancelado.")
            return
        if usuarios_db[nombre_usuario]['contraseña'] == contraseña:
            print("Login exitoso. ¡Bienvenido!")
            usuarios_db[nombre_usuario]['intentos'] = 0  # Borrar intentos fallidos después de login correcto
            usuario_actual = nombre_usuario
            guardar_datos(usuarios_db)
            return
        else:
            intentos_restantes -= 1
            usuarios_db[nombre_usuario]['intentos'] += 1
            print(f"Contraseña incorrecta. Intentos restantes: {intentos_restantes}")
            guardar_datos(usuarios_db)

        if intentos_restantes == 0:
            usuarios_db[nombre_usuario]['bloqueado'] = True
            guardar_datos(usuarios_db)
            print("Ha fallado 3 veces. Usuario Bloqueado.")
            return

In [34]:
def logout_usuario():
    global usuario_actual
    if usuario_actual:
        print(f"Sesión Finalizada.")
        usuario_actual = None
    else:
        print("No hay ninguna sesión iniciada.")

In [35]:
def desbloquear_usuario():
    nombre_usuario = input("Ingrese el nombre de usuario a desbloquear: ")
    if nombre_usuario in usuarios_db:
        if usuarios_db[nombre_usuario]['bloqueado']:
            usuarios_db[nombre_usuario]['bloqueado'] = False
            usuarios_db[nombre_usuario]['intentos'] = 0
            guardar_datos(usuarios_db)
            print(f"Usuario '{nombre_usuario}' desbloqueado.")
        else:
            print(f"El usuario '{nombre_usuario}' no está bloqueado.")
    else:
        print("El usuario no existe.")

In [36]:
def cambiar_contraseña():
    nombre_usuario = input("Ingrese su nombre de usuario: ")

    if nombre_usuario not in usuarios_db:
        print("El usuario no existe.")
        return

    contraseña_actual = input("Ingrese su contraseña actual: ")
    if usuarios_db[nombre_usuario]['contraseña'] == contraseña_actual:
        nueva_contraseña = input("Ingrese la nueva contraseña: ")
        usuarios_db[nombre_usuario]['contraseña'] = nueva_contraseña
        usuarios_db[nombre_usuario]['intentos'] = 0
        guardar_datos(usuarios_db)
        print(f"Contraseña del usuario '{nombre_usuario}' actualizada.")
    else:
        print("Contraseña actual incorrecta.")

In [37]:
def mostrar_datos():
    global usuario_actual
    if not usuario_actual or not usuarios_db[usuario_actual].get('admin', False):
        print("Acceso denegado. Solo un administrador puede ver los datos.")
        return

    print("\n--- Datos Guardados en la Base de Datos ---")
    for usuario, datos in usuarios_db.items():
        admin_texto = "Sí" if datos.get('admin', False) else "No"
        print(f"Usuario: {usuario}, Contraseña: {datos['contraseña']}, Bloqueado: {datos['bloqueado']}, Administrador: {admin_texto}")


In [ ]:
def mostrar_menu():
    print("\n--- Menú Principal ---")
    if not usuario_actual:
        print("1. Crear usuario")
        print("2. Login")
        print("3. Finalizar ejecución del programa")
    else:
        print("1. Logout")
        print("2. Cambiar contraseña")
        print("3. Finalizar ejecución del programa")
        if usuarios_db[usuario_actual].get('admin', False):
            print("4. Desbloquear usuario")
            print("5. Mostrar datos (Admin)")

while True:
    mostrar_menu()
    opcion = input("Seleccione una opción: ")

    if opcion == "1" and not usuario_actual:
        crear_usuario()
    elif opcion == "2" and not usuario_actual:
        login_usuario()
    elif opcion == "1" and usuario_actual:
        logout_usuario()
    elif opcion == "2" and usuario_actual:
        cambiar_contraseña()
    elif opcion == "3":
        logout_usuario()
        print("Finalizando ejecución del programa.")
        break
    elif opcion == "4" and usuario_actual and usuarios_db[usuario_actual].get('admin', False):
        desbloquear_usuario()
    elif opcion == "5" and usuario_actual and usuarios_db[usuario_actual].get('admin', False):
        mostrar_datos()
    else:
        print("Opción NO disponible para su perfil de Usuario. Intente nuevamente.")